In [1]:
library(INLA)
library(rSPDE)
library(readr)
library(fmesher)
library(inlabru)
data_ori <- as.data.frame(read_csv("tmp/1180_vol.csv"))
logpt<-log(data_ori$Close)
logreturn <-logpt[2:length(logpt)]-logpt[1:(length(logpt)-1)]
date <- data_ori$Date[1:(length(data_ori$Date)-1)]
   
date_timeindex<-as.numeric(as.POSIXct(date)) 
date_timeindex <- (date_timeindex - min(date_timeindex))/86400
mesh <- fm_mesh_1d(date_timeindex)
data = list(logreturn = logreturn, times = date_timeindex)


#OU model
OU_model <- rspde.matern(mesh = mesh, nu=0.5, prior.range.nominal = 50)
formula_OU <- logreturn ~ Intercept(1) +  OU(times, model = OU_model)
fit_OU <- bru(formula_OU, family="stochvol", data = data)

rspde_model <- rspde.matern(mesh = mesh, nu.upper.bound = 2, prior.range.nominal = 50)
formula_general <- logreturn ~ Intercept(1) + matern(times, model = rspde_model) 
fit_general <- bru(formula_general, family="stochvol", data = data)

train <- test <- list()
est.vol <- c() 
k = 1
for(i in seq(from = 10, to = length(date_timeindex)-10, by = 10)){
  train[[k]] <- 1:i
  test[[k]] <- (i+1):(i+10)
  est.vol <- c(est.vol, sqrt(var(logreturn[(i+1):(i+10)])))
  k = k+1
}


models <- list(fit_OU=fit_OU, fit_general=fit_general)

res2 <- cross_validation(models,train_test_indexes = list(train = train, test = test))


Warning message:
“package ‘INLA’ was built under R version 4.3.3”
Loading required package: Matrix



Loading required package: sp

This is INLA_24.04.01 built 2024-04-01 19:43:22 UTC.
 - See www.r-inla.org/contact-us for how to get help.
 - List available models/likelihoods/etc with inla.list.models()
 - Use inla.doc(<NAME>) to access documentation

Rows: 2372 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl  (9): Open, High, Low, Close, Volume, Dividends, Stock Splits, vol10, vol30
date (1): Date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Warning message in rspde.matern(mesh = mesh, nu = 0.5, prior.range.nominal = 50):
“In dimension 1 you can have unstable results
      for nu.upper.bound > 2. Consider changing
      nu.upper.bound to 2 or 1.”


Fold: 1 / 236 
Model: fit_OU 
Fold: 1 / 236 
Model: fit_general 
Fold: 2 / 236 
Model: fit_OU 
Fold: 2 / 236 
Model: fit_general 
Fold: 3 / 236 
Model: fit_OU 
Fold: 3 / 236 
Model: fit_general 
Fold: 4 / 236 
Model: fit_OU 
Fold: 4 / 236 
Model: fit_general 
Fold: 5 / 236 
Model: fit_OU 
Fold: 5 / 236 
Model: fit_general 
Fold: 6 / 236 
Model: fit_OU 
Fold: 6 / 236 
Model: fit_general 
Fold: 7 / 236 
Model: fit_OU 
Fold: 7 / 236 
Model: fit_general 
Fold: 8 / 236 
Model: fit_OU 
Fold: 8 / 236 
Model: fit_general 
Fold: 9 / 236 
Model: fit_OU 
Fold: 9 / 236 
Model: fit_general 
Fold: 10 / 236 
Model: fit_OU 
Fold: 10 / 236 
Model: fit_general 
Fold: 11 / 236 
Model: fit_OU 
Fold: 11 / 236 
Model: fit_general 
Fold: 12 / 236 
Model: fit_OU 
Fold: 12 / 236 
Model: fit_general 
Fold: 13 / 236 
Model: fit_OU 
Fold: 13 / 236 
Model: fit_general 
Fold: 14 / 236 
Model: fit_OU 
Fold: 14 / 236 
Model: fit_general 
Fold: 15 / 236 
Model: fit_OU 
Fold: 15 / 236 
Model: fit_general 
Fold: 16 / 23

In [6]:
packageVersion("rSPDE")

[1] ‘2.3.3’

In [3]:
res2

,Model,dss,mse,crps,scrps
,<chr>,<chr>,<chr>,<chr>,<chr>
1,fit_OU,0,0,0,0
2,fit_general,0,0,0,0
,Best,fit_OU,fit_OU,fit_OU,fit_OU


In [8]:
res2

,Model,dss,mse,crps,scrps
,<chr>,<chr>,<chr>,<chr>,<chr>
1,Model 1,0,0,0,0
,Best,Model 1,Model 1,Model 1,Model 1


In [3]:
cross_result[1]

,Model,dss,mse,crps,scrps
,<chr>,<chr>,<chr>,<chr>,<chr>
1,model,0,0,0,0
,Best,model,model,model,model
